## FOLD 1-5

In [ ]:
call_base = "ml-100k"

all_items = set(range(1, 1683))  # Semua item ID dari 1–1682
rated_counts = []
not_rated_counts = []

rated_any_folds = set()
not_rated_any_folds = set()
rated_all_folds = None
not_rated_all_folds = None

for i in range(1, 6):
    basedata, testdata = load_Dataset(call_base, i)
    rating_matriks = ConvertRatingMatriks(basedata)
    
    # Item yang sudah dirating
    itemRated = set(rating_matriks.columns[rating_matriks.sum(axis=0) > 0])
    print(f"item rated {len(itemRated)}")
    
    # Item yang belum dirating
    itemNotRated = all_items - itemRated
    print(f"item  all rated {len(all_items)}")
    
    # Simpan info per fold
    rated_counts.append((i, len(itemRated)))
    not_rated_counts.append((i, len(itemNotRated)))
    
    # Gabungan dan irisan
    rated_any_folds.update(itemRated)
    not_rated_any_folds.update(itemNotRated)
    
    if rated_all_folds is None:
        rated_all_folds = itemRated
        not_rated_all_folds = itemNotRated
    else:
        rated_all_folds = rated_all_folds.intersection(itemRated)
        not_rated_all_folds = not_rated_all_folds.intersection(itemNotRated)
    
    print(f"Fold {i}:")
    print(f"- Item sudah dirating: {len(itemRated)}")
    print(f"- Item belum dirating : {len(itemNotRated)}\n")



In [1]:
# import

import pandas as pd 
import numpy as np 
import math as math
import joblib
from joblib import dump
import os
import time

## PREDIKSI

In [2]:
# def TetanggaK(target_user, target_item, matriks_rating, similarity, k, jenis="user-based"):
#     rating_matrix = matriks_rating.to_numpy()  # Convert ke NumPy untuk efisiensi
#     if jenis == "user-based":
#         # mendapatkan rating dari user yang ditargetkan
#         item_ratings = rating_matrix[:, target_item]
#         # mencari item yang sudah di rating oleh user
#         item_rated_by = np.flatnonzero(item_ratings)
#         # print(f"item rated by {len(item_rated_by)}")
#         item_rated_by = item_rated_by[item_rated_by != target_user]
#         # ambil valaue similarity dari user yang ditargetkan ke item yang sudah di rating oleh user
#         sim_scores = similarity.values[target_user, item_rated_by]
#         indices = item_rated_by
#     else:  # item-based
#         user_ratings = rating_matrix[target_user, :]
#         items_rated = np.flatnonzero(user_ratings)
#         items_rated = items_rated[items_rated != target_item]
#         sim_scores = similarity.values[items_rated, target_item]
#         indices = items_rated
#     #diurutka nilai terbesar ke terkecil
#     sorted_idx = np.argsort(-sim_scores)[:k]
#     top_neighbors = [indices[i] for i in sorted_idx]
#     return top_neighbors


In [3]:
# def prediksiCF_v2(RatingMatriks, similarityFunction, mean, meanCen, user=2, item=2, k=2, jenis="user-based"):
#     # jika jenis user-based
#     if jenis == "user-based":
#         target_user1 = user
#         target_item2 = item
#         # ambil mean rating user target
#         mean_value = mean.iloc[user]
#         # print("mean", mean_value)
#     elif jenis == "item-based":
#         target_user1 = item
#         target_item2 = user
#         # ambil mean rating item target
#         mean_value = mean.iloc[item]
#     # Ambil top-k tetangga
#     tetangga = TetanggaK(
#         target_user=target_user1,
#         target_item=target_item2,
#         matriks_rating=RatingMatriks,
#         similarity=similarityFunction,
#         k=k,
#         jenis=jenis
#     )
#     # print(f"tetangga {len(tetangga)}")
#     # menyiapkan pembilang dan penyebut untuk menghitung prediksi rating
#     pembilang = 0
#     penyebut = 0
#     # loop sesuai dengan banyak tetangga
#     for tetangga_idx in tetangga:
#         if jenis == "user-based":
#             # mengambil sim antara user dan tetangga
#             sim = similarityFunction.iloc[user, tetangga_idx]
#             # print(f"sim {user}-{tetangga_idx} = {sim}")
#             mean_cen = meanCen.iloc[tetangga_idx, item]
#             # print(f"mean_cen {tetangga_idx}-{item} = {mean_cen}")
#         else:  # item-based
#             # mengambil sim antara item dan tetangga
#             sim = similarityFunction.iloc[tetangga_idx, item]
#             # mengambil mean centered rating untuk user target dari tetangga
#             mean_cen = meanCen.iloc[user, tetangga_idx]
#         pembilang += mean_cen * sim
#         # print("pembilang", (pembilang))
#         # menghitung penyebut
#         penyebut += abs(sim)
#         # print("penyebut", penyebut)
#     # jika penyebut == 0, maka hitung predikis
#     if penyebut != 0:
#         prediksi = mean_value + (pembilang / penyebut)
#         # print(f"prediks1 {prediksi} = {mean_value} + ({pembilang} / {penyebut})")
#     else:
#         prediksi = mean_value
#     return [item, float(prediksi)]

In [4]:
# def prediksi_semua_matriks(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
#     # Salin matriks rating agar tidak mengubah data asli
#     hasil_matriks = RatingMatriks.copy()
#     num_users, num_items = hasil_matriks.shape
#     for user in range(num_users):
#         for item in range(num_items):
#             if hasil_matriks.iloc[user, item] == 0:
#                 _, pred_rating = prediksiCF_v2(
#                     RatingMatriks=RatingMatriks,
#                     similarityFunction=similarityFunction,
#                     mean=mean,
#                     meanCen=meanCen,
#                     user=user,
#                     item=item,
#                     k=k,
#                     jenis=jenis
#                 )
#                 hasil_matriks.iloc[user, item] = pred_rating
#     return hasil_matriks

In [5]:
import numpy as np
import pandas as pd

def TetanggaK4(target_user, target_item, rating_matrix_np, similarity_np, k, jenis="user-based"):
    """
    Mengembalikan indeks K tetangga terdekat (user atau item) berdasarkan similarity.
    """
    if jenis == "user-based":
        # Ambil semua rating terhadap item target
        item_ratings = rating_matrix_np[:, target_item]
        # User yang pernah memberi rating terhadap item tersebut (kecuali user target)
        tetangga_idx = np.where((item_ratings != 0) & (np.arange(len(item_ratings)) != target_user))[0]
        # Ambil skor similarity user-user
        sim_scores = similarity_np[target_user, tetangga_idx]
    else:  # item-based
        # Di item-based, user tetap user, item adalah target
        user_ratings = rating_matrix_np[target_user, :]
        # Cari item lain yang pernah diberi rating oleh user target (selain item target)
        tetangga_idx = np.where((user_ratings != 0) & (np.arange(rating_matrix_np.shape[1]) != target_item))[0]
        # Ambil skor similarity item-item
        sim_scores = similarity_np[tetangga_idx, target_item]

    if len(sim_scores) == 0:
        return []

    # Urutkan tetangga berdasarkan similarity tertinggi
    sorted_idx = np.argsort(-sim_scores)[:k]
    return tetangga_idx[sorted_idx]


def prediksiCF_v2_optimized(RatingMatriks, similarityFunction, mean, meanCen, user, item, k=2, jenis="user-based"):
    """
    Menghitung prediksi rating menggunakan Collaborative Filtering berbasis user/item, versi optimal.
    """

    # Cache ke bentuk NumPy
    rating_np = RatingMatriks.values
    similarity_np = similarityFunction.values
    mean_np = mean.values
    meanCen_np = meanCen.values

    # Set indeks untuk pencarian tetangga
    target_user = user
    target_item = item
    mean_value = mean_np[user] if jenis == "user-based" else mean_np[item]

    # Cari tetangga
    tetangga = TetanggaK4(target_user, target_item, rating_np, similarity_np, k, jenis)

    if len(tetangga) == 0:
        return float(mean_value)

    # Ambil similarity dan mean centered rating
    if jenis == "user-based":
        sim = similarity_np[user, tetangga]
        mean_cen = meanCen_np[tetangga, item]
    else:  # item-based
        sim = similarity_np[tetangga, item]
        mean_cen = meanCen_np[user, tetangga]

    pembilang = np.dot(mean_cen, sim)
    penyebut = np.sum(np.abs(sim))

    prediksi = mean_value + (pembilang / penyebut) if penyebut != 0 else mean_value
    return float(prediksi)


def prediksi_semua_matriks2(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="user-based"):
    """
    Mengisi semua rating kosong dalam matriks dengan hasil prediksi CF.
    """
    hasil_matriks = RatingMatriks.copy()
    num_users, num_items = hasil_matriks.shape

    for user in range(num_users):
        for item in range(num_items):
            if hasil_matriks.iloc[user, item] == 0:
                pred_rating = prediksiCF_v2_optimized(
                    RatingMatriks=RatingMatriks,
                    similarityFunction=similarityFunction,
                    mean=mean,
                    meanCen=meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis
                )
                hasil_matriks.iloc[user, item] = pred_rating
    return hasil_matriks


In [6]:
def simpan_semua_prediksi_k(
    RatingMatriks, similarityFunction, mean, meanCen,
    variasi_k, jenis="item-based",
    folder_output="../case/prediksiModel/Jac/ub/2"
):
    for k in variasi_k:
        # Hitung prediksi
        prediksi = prediksi_semua_matriks2(
            RatingMatriks=RatingMatriks,
            similarityFunction=similarityFunction,
            mean=mean,
            meanCen=meanCen,
            k=k,
            jenis=jenis
        )

        # Siapkan path file
        path_file = os.path.join(folder_output, f"{k}.joblib")
        os.makedirs(folder_output, exist_ok=True)

        # Simpan hasil prediksi
        joblib.dump(prediksi, path_file)
        print(f"✔️ Prediksi untuk k={k} disimpan di: {path_file}")

In [ ]:
ratingMatriks = joblib.load("../case/ratingMatriks/u5.joblib")
similartias = joblib.load("../case/sim/relevantJ/ib/simRJFold5.joblib")
mean = joblib.load("../case/mean/ib/meanFold5.joblib")
meanCen = joblib.load("../case/meanCen/ib/meanCen5.joblib")

In [8]:
# [5, 10, 15, 18, 20, 25, 30, 40, 50, 100, 200],

In [14]:
simpan_semua_prediksi_k(
    RatingMatriks=ratingMatriks,
    similarityFunction=similartias,
    mean=mean,
    meanCen=meanCen,
    variasi_k=[100, 200],
    jenis="item-based",
    folder_output="../case/prediksiModel/RJ/ib/5"
)

✔️ Prediksi untuk k=100 disimpan di: ../case/prediksiModel/RJ/ib/5\100.joblib
✔️ Prediksi untuk k=200 disimpan di: ../case/prediksiModel/RJ/ib/5\200.joblib


In [ ]:
calPredksiJacIb = joblib.load("../case/prediksiModel/Jac/ib/1/5.joblib")
calPredksiJacIb

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.000000,3.000000,4.000000,3.000000,3.000000,3.726984,4.000000,1.000000,5.000000,4.166377,...,2.337392,4.875795,2.716800,1.716800,3.215376,1.647888,3.647888,2.647888,3.361404,2.973617
2,4.000000,3.715148,3.345578,4.085283,3.520709,3.495320,4.347754,4.682421,4.314873,2.000000,...,3.154477,4.538400,2.804315,1.804315,2.889400,1.179907,3.179907,2.179907,3.118929,3.544333
3,3.080438,2.212793,2.136147,2.140093,2.705722,2.770453,2.683748,2.892266,2.531532,2.304081,...,1.454103,3.427865,2.191256,1.191256,3.511935,0.775611,2.775611,1.775611,3.241623,1.869708
4,4.156999,3.148092,3.079839,3.775742,3.481321,4.765514,4.229108,4.365522,4.139582,4.050432,...,2.066421,4.097364,3.882692,2.882692,3.511561,2.747439,4.747439,3.747439,3.351349,3.122463
5,3.969435,3.422603,3.180199,3.120668,2.976296,2.243981,3.872844,3.449859,4.146571,3.174323,...,1.078572,4.426300,2.177464,1.177464,2.285240,1.000000,3.000000,2.000000,2.331302,3.080433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,5.232625,4.250524,4.027182,4.593374,4.304039,4.138492,5.038734,5.123288,5.000000,4.800196,...,4.062040,5.135692,3.557936,2.557936,4.123034,2.266218,4.266218,3.266218,4.191240,3.765144
940,3.315556,3.071910,2.786752,2.000000,2.648929,2.346380,4.000000,5.000000,3.000000,3.048660,...,2.020164,4.146958,1.656058,0.656058,2.653414,0.709038,2.709038,1.709038,2.897039,1.360398
941,5.000000,4.061256,3.052515,3.949792,3.381664,3.806699,4.000000,4.431505,4.109140,3.983205,...,3.503131,4.582748,3.506920,2.506920,3.629370,1.449405,3.449405,2.449405,3.205345,3.532239
942,4.555486,4.331408,3.733854,4.086524,3.659604,3.984275,4.455978,4.902842,4.416447,3.957937,...,3.798138,4.381947,3.815630,2.815630,3.457255,1.282419,3.282419,2.282419,3.493898,3.984387


In [ ]:
print(f"similarits {similartias.shape}")
print(f"rating {ratingMatriks.shape}")
print(f"mean {mean.shape}")
print(f"mean cen {meanCen.shape}")

similarits (1682, 1682)
rating (943, 1682)
mean (1682, 1)
mean cen (943, 1682)


In [ ]:
5, 10, 15, 18, 20, 25, 30, 40, 50, 100,

(5, 10, 15, 18, 20, 25, 30, 40, 50, 100)